# Versión antigua (ocultar; no usar)

Esta celda contiene el código anterior para simular la situación base. Se mantuvo por si los cambios hechos en la celda siguiente provocan algún error desconocido. Se modificó la función y tiene cambios que no existen en el commit anterior; es por esto que se mantiene.

In [34]:
'''
import json
import pandas as pd
import random
import heapq
from collections import deque, defaultdict
from datetime import datetime
import uuid
from pathlib import Path

class Node:
    def __init__(self, name, mu, servers):
        self.name = name
        self.mu = mu # tasa de servicio de cada servidor
        self.servers = servers # número de servidores
        self.busy = 0 # servidores ocupados
        self.queue = deque() # cola de clientes (pid, tiempo_de_llegada_a_cola)
        # estadísticas
        self.area_q = 0.0 # integral de longitud de cola
        self.area_busy = 0.0 # integral de servidores ocupados
        self.last_event_time = 0.0
        self.num_served = 0 # número de clientes servidos

class Event:
    def __init__(self, time, kind, node_name, pid, external=False):
        self.time = time # instante del evento
        self.kind = kind # 'arrival' o 'departure'
        self.node_name = node_name # nodo donde ocurre
        self.pid = pid # identificador del cliente
        self.external = external # si es llegada externa
    def __lt__(self, other):
        return self.time < other.time

def simulate(T: float, lambda1: float, lambda2: float, nodes: dict[str, Node]
        ) -> dict[str, dict[str, float]]:
    """
    Args:
        T: Tiempo de simulación (horas).
        lambda1: Tasa de llegada externa a 'reg1' (pacientes/hora).
        lambda2: Tasa de llegada externa a 'reg2' (pacientes/hora).
        nodes: Diccionario <nombre, Node>
    
    Returns:
        metrics: Diccionario <nombre_nodo, dict<nombre_metrica, float>>
            Explicación de las métricas:
            - utilización: Proporción de tiempo ocupado por los servidores.
            - Lq: Longitud promedio de la cola (número de clientes en cola).
            - Wq: Tiempo promedio en cola (horas).
            - throughput: Rendimiento (número de clientes atendidos por hora).
    """
    # lista de eventos
    future_events = []
    t = 0.0
    pid_counter = 0

    # estadísticas de espera por nodo
    waits = defaultdict(list)

    # Programar primeras llegadas externas

    # lambda 1 y lambda 2 son tasas de llegada por hora
    # y el tiempo de llegada es exponencial con parámetro lambda
    # (1/lambda es el tiempo promedio entre llegadas)
    # Por lo tanto, la llegada de un cliente es un evento que ocurre a
    # t + expovariate(lambda) horas
    
    # Los primeros eventos son llegadas externas a los nodos
    # reg1 y reg2, que son los únicos nodos con llegadas externas
    # Aquí, el time de los eventos es t + expovariate(lambda1) y t + expovariate(lambda2),
    # donde t es el tiempo actual (inicialmente 0, así que sólo se pasa lambda a random.expovariate)
    heapq.heappush(future_events, Event(random.expovariate(lambda1), 'arrival', 'reg1', pid_counter, True))
    pid_counter += 1
    heapq.heappush(future_events, Event(random.expovariate(lambda2), 'arrival', 'reg2', pid_counter, True))
    pid_counter += 1

    while future_events:
        event = heapq.heappop(future_events)
        if event.time > T:
            break
        # avanzamos el reloj
        prev_t = t
        t = event.time
        # actualizamos áreas bajo Q y busy para cada nodo
        for nd in nodes.values():
            dt = t - nd.last_event_time
            nd.area_q += len(nd.queue) * dt
            nd.area_busy += nd.busy * dt
            nd.last_event_time = t

        node = nodes[event.node_name]

        if event.kind == 'arrival':
            # si llegó externamente, programar siguiente llegada externa
            if event.external:
                if event.node_name == 'reg1':
                    heapq.heappush(future_events, Event(t + random.expovariate(lambda1),
                                           'arrival', 'reg1', pid_counter, True))
                else:
                    heapq.heappush(future_events, Event(t + random.expovariate(lambda2),
                                           'arrival', 'reg2', pid_counter, True))
                pid_counter += 1

            # llegada al nodo: si hay servidor libre se atiende enseguida
            if node.busy < node.servers:
                node.busy += 1
                waits[node.name].append(0.0)
                # se programa la salida
                dt = random.expovariate(node.mu)
                heapq.heappush(future_events, Event(t + dt, 'departure', node.name, event.pid))
            else:
                # se pone en cola
                node.queue.append((event.pid, t))

        else: # departure
            node.num_served += 1
            node.busy -= 1
            # si hay cola, empieza el siguiente
            if node.queue:
                pid_q, t_arr_q = node.queue.popleft()
                wait_time = t - t_arr_q
                waits[node.name].append(wait_time)
                node.busy += 1
                dt = random.expovariate(node.mu)
                heapq.heappush(future_events, Event(t + dt, 'departure', node.name, pid_q))

            # determinar a dónde va el cliente
            if event.node_name == 'reg1':
                next_node = 'exam1'
            elif event.node_name == 'reg2':
                next_node = 'exam2'
            elif event.node_name.startswith('exam'):
                # aleatorio 50/50
                next_node = 'consult1' if random.random() < 0.5 else 'consult2'
            elif event.node_name.startswith('consult'):
                next_node = 'pharma1' if event.node_name == 'consult1' else 'pharma2'
            else:
                # en farmacia sale del sistema
                continue

            # llegada instantánea al siguiente nodo
            heapq.heappush(future_events, Event(t, 'arrival', next_node, event.pid, False))

    # métricas de desempeño
    metrics = {}
    for name, nd in nodes.items():
        Lq = nd.area_q / T
        rho = nd.area_busy / T / nd.servers
        Wq = (sum(waits[name]) / len(waits[name])) if waits[name] else 0.0
        throughput = nd.num_served / T
        metrics[name] = {
            'Wq': Wq,
            'Lq': Lq,
            'rho': rho,
            'throughput': throughput,
        }
    return metrics

if __name__ == '__main__':
    rand_seed = 1
    random.seed(rand_seed)
    print(f"Semilla aleatoria: {rand_seed}")

    # Definir nodos: name, mu, servers
    nodes = {
        'reg1': Node('reg1', mu=10.0, servers=1),
        'reg2': Node('reg2', mu=10.0, servers=1),
        'exam1': Node('exam1', mu=6.0, servers=1),
        'exam2': Node('exam2', mu=6.0, servers=1),
        'consult1':Node('consult1', mu=4.0, servers=2),
        'consult2':Node('consult2', mu=4.5, servers=2),
        'pharma1': Node('pharma1', mu=12.0, servers=1),
        'pharma2': Node('pharma2', mu=10.0, servers=1),
    }

    # Parámetros de llegada externa (pacientes/hora)
    lambda1 = 5.0 # a reg1
    lambda2 = 3.0 # a reg2

    T = 500.0 # horas de simulación

    results = simulate(T, lambda1, lambda2, nodes)

    # Mostrar resultados
    # Convert simulation results into a DataFrame and print as a table
    df = pd.DataFrame(results).T
    df = df[['Wq', 'Lq', 'rho', 'throughput']]
    # Change number of decimals to display to 3
    df = df.round(3)
    display(df)

    # Create a unique directory under "runs"
    timestamp = datetime.now().strftime("%d-%m-%Y %H_%M_%S")
    unique_id = uuid.uuid4().hex[:6]  # Short unique identifier
    run_path = Path("runs") / "base_original" / f"{timestamp} {unique_id}"
    run_path.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    csv_path = run_path / 'simulation_results.csv'
    df.to_csv(csv_path, index=True)
    print(f"Resultados guardados en '{csv_path}'.")

    # Save random seed, nodes and parameters to a JSON file
    json_path = run_path / 'simulation_params.json'
    with json_path.open('w') as f:
        json.dump({
            'rand_seed': rand_seed,
            'nodes': {name: {'mu': node.mu, 'servers': node.servers} for name, node in nodes.items()},
            'lambda1': lambda1,
            'lambda2': lambda2,
            'T': T
        }, f, indent=4)
    print(f"Resultados guardados en '{json_path}'.")
'''

'\nimport json\nimport pandas as pd\nimport random\nimport heapq\nfrom collections import deque, defaultdict\nfrom datetime import datetime\nimport uuid\nfrom pathlib import Path\n\nclass Node:\n    def __init__(self, name, mu, servers):\n        self.name = name\n        self.mu = mu # tasa de servicio de cada servidor\n        self.servers = servers # número de servidores\n        self.busy = 0 # servidores ocupados\n        self.queue = deque() # cola de clientes (pid, tiempo_de_llegada_a_cola)\n        # estadísticas\n        self.area_q = 0.0 # integral de longitud de cola\n        self.area_busy = 0.0 # integral de servidores ocupados\n        self.last_event_time = 0.0\n        self.num_served = 0 # número de clientes servidos\n\nclass Event:\n    def __init__(self, time, kind, node_name, pid, external=False):\n        self.time = time # instante del evento\n        self.kind = kind # \'arrival\' o \'departure\'\n        self.node_name = node_name # nodo donde ocurre\n        se

# Simulación de situación base y de posibles soluciones

In [35]:
import json
import pandas as pd
import random
import heapq
from collections import deque, defaultdict
from datetime import datetime
import uuid
from pathlib import Path

class Node:
    def __init__(self, name, mu, servers):
        self.name = name
        self.mu = mu # tasa de servicio de cada servidor
        self.servers = servers # número de servidores
        self.busy = 0 # servidores ocupados
        self.queue = deque() # cola de clientes (pid, tiempo_de_llegada_a_cola)
        # estadísticas
        self.area_q = 0.0 # integral de longitud de cola
        self.area_busy = 0.0 # integral de servidores ocupados
        self.last_event_time = 0.0
        self.num_served = 0 # número de clientes servidos

class Event:
    def __init__(self, time, kind, node_name, pid, external=False):
        self.time = time # instante del evento
        self.kind = kind # 'arrival' o 'departure'
        self.node_name = node_name # nodo donde ocurre
        self.pid = pid # identificador del cliente
        self.external = external # si es llegada externa
    def __lt__(self, other):
        return self.time < other.time

## Función para simular situación base (antes de propuestas de soluciones a problemas)

In [36]:
def simulate_base(T: float, lambda1: float, lambda2: float, nodes: dict[str, Node]
        ) -> dict[str, dict[str, float]]:
    """
    Simulación original con dos salas de registro independientes (reg1, reg2).
    """
    future = []
    t = 0.0
    pid_ctr = 0
    waits = defaultdict(list)

    # primeras llegadas externas a reg1 y reg2
    heapq.heappush(future, Event(random.expovariate(lambda1), 'arrival', 'reg1', pid_ctr, True))
    pid_ctr += 1
    heapq.heappush(future, Event(random.expovariate(lambda2), 'arrival', 'reg2', pid_ctr, True))
    pid_ctr += 1

    while future:
        ev = heapq.heappop(future)
        if ev.time > T:
            break
        # avanzar reloj y actualizar áreas
        dt_global = ev.time - t
        t = ev.time
        for nd in nodes.values():
            nd.area_q    += len(nd.queue)*dt_global
            nd.area_busy += nd.busy*dt_global
            nd.last_event_time = t

        node = nodes[ev.node_name]
        if ev.kind == 'arrival':
            # reprogramar siguiente llegada externa
            if ev.external:
                lam = lambda1 if ev.node_name=='reg1' else lambda2
                heapq.heappush(future, Event(t+random.expovariate(lam),
                                             'arrival', ev.node_name, pid_ctr, True))
                pid_ctr += 1
            # si hay servidor libre
            if node.busy < node.servers:
                node.busy += 1
                waits[node.name].append(0.0)
                dt = random.expovariate(node.mu)
                heapq.heappush(future, Event(t+dt, 'departure', node.name, ev.pid))
            else:
                node.queue.append((ev.pid, t))

        else:  # departure
            node.num_served += 1
            node.busy -= 1
            # si hay cola, atiende siguiente
            if node.queue:
                pid_q, t_arr = node.queue.popleft()
                waits[node.name].append(t - t_arr)
                node.busy += 1
                dt = random.expovariate(node.mu)
                heapq.heappush(future, Event(t+dt, 'departure', node.name, pid_q))
            # ruta al siguiente nodo
            if ev.node_name == 'reg1':
                nxt = 'exam1'
            elif ev.node_name == 'reg2':
                nxt = 'exam2'
            elif ev.node_name.startswith('exam'):
                nxt = 'consult1' if random.random()<0.5 else 'consult2'
            elif ev.node_name.startswith('consult'):
                nxt = 'pharma1' if ev.node_name=='consult1' else 'pharma2'
            else:
                continue
            # llegada inmediata
            heapq.heappush(future, Event(t, 'arrival', nxt, ev.pid, False))

    # calcular métricas
    metrics = {}
    for name, nd in nodes.items():
        Lq = nd.area_q / T
        rho = nd.area_busy / T / nd.servers
        Wq = sum(waits[name]) / len(waits[name]) if waits[name] else 0.0
        X   = nd.num_served / T
        metrics[name] = {
            'Wq': Wq,
            'Lq': Lq,
            'rho': rho,
            'throughput': X
        }
    return metrics

## Función para simular situación de la solución 1

Actualmente, se tienen dos salas de registro separadas como estaciones de servicio independientes con colas separadas e independientes. 

**Esta solución 1 consiste en modificar el sistema de colas de la siguiente manera**: Una sola sala de registro con 2 servidores y una única cola. La sala de registro sería un modelo M/M/2 con tasa de llegada lambda = lambda1 + lambda2.

Si son dos M/M/1 independientes, cada cola se comporta aisladamente.

Si es un M/M/2 con cola compartida, se consigue, para la misma capacidad total, menores tiempos de espera promedio (la famosa ganancia del pooling).

"It is generally accepted that operating with a combined (i.e., pooled) queue rather than separate (i.e., dedicated) queues is beneficial mainly because pooling queues reduces long-run average throughput time" (Sunar, N., 2018, recuperado de: https://kenaninstitute.unc.edu/publication/pooled-or-dedicated-queues-when-customers-are-delay-sensitive/).


In [37]:
import copy

def simulate_pooled(T: float, lambda1: float, lambda2: float, nodes: dict[str, Node]
        ) -> dict[str, dict[str, float]]:
    """
    Versión mejorada: un único nodo 'reg' con 2 servidores y cola compartida.
    Las llegadas externas a 'reg' ocurren a tasa (lambda1+lambda2).
    Al terminar el registro, se asigna a exam1/exam2 con prob. proporcional
    a lambda1/(lambda1+lambda2) y lambda2/(lambda1+lambda2).
    """
    future = []
    t = 0.0
    pid_ctr = 0
    waits = defaultdict(list)
    lam_tot = lambda1 + lambda2

    # primera llegada externa a 'reg'
    heapq.heappush(future, Event(random.expovariate(lam_tot), 'arrival', 'reg', pid_ctr, True))
    pid_ctr += 1

    while future:
        ev = heapq.heappop(future)
        if ev.time > T:
            break
        # avanzar reloj y actualizar áreas
        dt_global = ev.time - t
        t = ev.time
        for nd in nodes.values():
            nd.area_q    += len(nd.queue)*dt_global
            nd.area_busy += nd.busy*dt_global
            nd.last_event_time = t

        node = nodes[ev.node_name]
        if ev.kind == 'arrival':
            # reprogramar siguiente llegada externa (solo en 'reg')
            if ev.external:
                heapq.heappush(future, Event(t+random.expovariate(lam_tot),
                                             'arrival', 'reg', pid_ctr, True))
                pid_ctr += 1
            # si hay servidor libre
            if node.busy < node.servers:
                node.busy += 1
                waits[node.name].append(0.0)
                dt = random.expovariate(node.mu)
                heapq.heappush(future, Event(t+dt, 'departure', node.name, ev.pid))
            else:
                node.queue.append((ev.pid, t))

        else:  # departure
            node.num_served += 1
            node.busy -= 1
            # si hay cola, atiende siguiente
            if node.queue:
                pid_q, t_arr = node.queue.popleft()
                waits[node.name].append(t - t_arr)
                node.busy += 1
                dt = random.expovariate(node.mu)
                heapq.heappush(future, Event(t+dt, 'departure', node.name, pid_q))
            # ruta al siguiente nodo
            if ev.node_name == 'reg':
                # asignar proporcionalmente
                p1 = lambda1 / lam_tot
                nxt = 'exam1' if random.random() < p1 else 'exam2'
            elif ev.node_name.startswith('exam'):
                nxt = 'consult1' if random.random()<0.5 else 'consult2'
            elif ev.node_name.startswith('consult'):
                nxt = 'pharma1' if ev.node_name=='consult1' else 'pharma2'
            else:
                continue
            # llegada inmediata
            heapq.heappush(future, Event(t, 'arrival', nxt, ev.pid, False))

    # calcular métricas
    metrics = {}
    for name, nd in nodes.items():
        Lq = nd.area_q / T
        rho = nd.area_busy / T / nd.servers
        Wq = sum(waits[name]) / len(waits[name]) if waits[name] else 0.0
        X   = nd.num_served / T
        metrics[name] = {
            'Wq': Wq,
            'Lq': Lq,
            'rho': rho,
            'throughput': X
        }
    return metrics



## Ejecutar simulaciones

### Entradas (*modificable por el usuario*)

In [38]:
# Para obtener exactamente los mismos resultados de una run en específico, 
# sólo usa la misma semilla, mismos parámetros y mismos nodos.

# Horas de simulación
T = 500.0
seed = random.randint(1, 100000) # semilla aleatoria

# Parámetros de llegada externa (pacientes/hora)
lambda1 = 4.0 # a reg1
lambda2 = 3.0 # a reg2

# 1) Modelo base: dos salas reg1, reg2
nodes_base = {
    'reg1':    Node('reg1',     mu=10.0, servers=1),
    'reg2':    Node('reg2',     mu=10.0, servers=1),
    'exam1':   Node('exam1',    mu=6.0,  servers=1),
    'exam2':   Node('exam2',    mu=6.0,  servers=1),
    'consult1':Node('consult1', mu=4.0,  servers=2),
    'consult2':Node('consult2', mu=4.5,  servers=2),
    'pharma1': Node('pharma1',  mu=12.0, servers=1),
    'pharma2': Node('pharma2',  mu=10.0, servers=1),
}

# 2) Modelo solución 1: un único reg con 2 servidores
nodes_solution1 = {
    'reg':     Node('reg',      mu=10.0, servers=2),
    'exam1':   Node('exam1',    mu=6.0,  servers=1),
    'exam2':   Node('exam2',    mu=6.0,  servers=1),
    'consult1':Node('consult1', mu=4.0,  servers=2),
    'consult2':Node('consult2', mu=4.5,  servers=2),
    'pharma1': Node('pharma1',  mu=12.0, servers=1),
    'pharma2': Node('pharma2',  mu=10.0, servers=1),
}

### Correr las simulaciones

In [39]:
random.seed(seed)
mb = simulate_base(T, lambda1, lambda2, copy.deepcopy(nodes_base))
random.seed(seed)
mp = simulate_pooled(T, lambda1, lambda2, copy.deepcopy(nodes_solution1))

# Agregar métrica de registro en el modelo base
Wq_reg_base = (lambda1*mb['reg1']['Wq'] + lambda2*mb['reg2']['Wq'])/(lambda1+lambda2)
Lq_reg_base = mb['reg1']['Lq'] + mb['reg2']['Lq']
util_reg_base = (mb['reg1']['rho'] + mb['reg2']['rho'])/2

# Crear DataFrame para el registro
df_reg = pd.DataFrame({
    'Wq base': [Wq_reg_base],
    'Wq pool': [mp['reg']['Wq']],
    'Lq base': [Lq_reg_base],
    'Lq pool': [mp['reg']['Lq']],
    'rho base': [util_reg_base],
    'rho pool': [mp['reg']['rho']]
}, index=['reg'])

print("\n=== Comparativa registro ===")
display(df_reg)

# Crear DataFrame para el resto de nodos
resto = ['exam1', 'exam2', 'consult1', 'consult2', 'pharma1', 'pharma2']
rows = []
for n in resto:
    rows.append({
        'Nodo': n,
        'Wq base': mb[n]['Wq'],
        'Wq pool': mp[n]['Wq'],
        'Lq base': mb[n]['Lq'],
        'Lq pool': mp[n]['Lq'],
        'rho base': mb[n]['rho'],
        'rho pool': mp[n]['rho']
    })
df_resto = pd.DataFrame(rows).set_index('Nodo')
print("\n=== Comparativa resto de nodos ===")
display(df_resto)

# Crear un directorio único dentro de "runs"
timestamp = datetime.now().strftime("%d-%m-%Y %H_%M_%S")
unique_id = uuid.uuid4().hex[:6] # ID único

# Crear la carpeta común usando timestamp y unique_id
base_folder = Path("runs") / f"{timestamp} {unique_id}"
base_folder.mkdir(parents=True, exist_ok=True)

# ------------------------------
# Guardar los resultados de la simulación base en runs/{timestamp} {unique_id}/base/
run_path_base = base_folder / "base"
run_path_base.mkdir(parents=True, exist_ok=True)

df_base = pd.DataFrame(mb).T
df_base = df_base[['Wq', 'Lq', 'rho', 'throughput']].round(3)
csv_path_base = run_path_base / 'simulation_results.csv'
df_base.to_csv(csv_path_base)
print(f"Resultados de la simulación base guardados en '{csv_path_base}'.")

params_base = {
    'seed': seed,
    'nodes': {name: {'mu': node.mu, 'servers': node.servers} for name, node in nodes_base.items()},
    'lambda1': lambda1,
    'lambda2': lambda2,
    'T': T,
}
json_path_base = run_path_base / 'simulation_params.json'
with json_path_base.open('w') as f:
    json.dump(params_base, f, indent=4)
print(f"Parámetros de la simulación base guardados en '{json_path_base}'.")

# ------------------------------
# Guardar los resultados de la simulación de la solución 1 en 
# runs/{timestamp} {unique_id}/solution1/
run_path_solution = base_folder / "solution1"
run_path_solution.mkdir(parents=True, exist_ok=True)

df_pool = pd.DataFrame(mp).T
df_pool = df_pool[['Wq', 'Lq', 'rho', 'throughput']].round(3)
csv_path_pool = run_path_solution / 'simulation_results.csv'
df_pool.to_csv(csv_path_pool)
print(f"Resultados de la simulación solución 1 guardados en '{csv_path_pool}'.")

params_pool = {
    'seed': seed,
    'nodes': {name: {'mu': node.mu, 'servers': node.servers} for name, node in nodes_solution1.items()},
    'lambda1': lambda1,
    'lambda2': lambda2,
    'T': T,
}
json_path_pool = run_path_solution / 'simulation_params.json'
with json_path_pool.open('w') as f:
    json.dump(params_pool, f, indent=4)
print(f"Parámetros de la simulación solución 1 guardados en '{json_path_pool}'.")


=== Comparativa registro ===


,Wq base,Wq pool,Lq base,Lq pool,rho base,rho pool
reg,0.056619,0.014846,0.398553,0.105198,0.356461,0.357135



=== Comparativa resto de nodos ===


,Wq base,Wq pool,Lq base,Lq pool,rho base,rho pool
Nodo,,,,,,
exam1,0.355140,0.408846,1.429083,1.655008,0.658058,0.696289
exam2,0.166349,0.148798,0.500379,0.452049,0.503264,0.506700
consult1,0.084803,0.076273,0.304782,0.279158,0.483675,0.473475
consult2,0.026622,0.037039,0.091473,0.126822,0.370799,0.376957
pharma1,0.035717,0.033013,0.128297,0.120829,0.292900,0.296661
pharma2,0.050656,0.059585,0.173851,0.203900,0.358199,0.342185


Resultados de la simulación base guardados en 'runs\08-05-2025 03_44_44 485d37\base\simulation_results.csv'.
Parámetros de la simulación base guardados en 'runs\08-05-2025 03_44_44 485d37\base\simulation_params.json'.
Resultados de la simulación solución 1 guardados en 'runs\08-05-2025 03_44_44 485d37\solution1\simulation_results.csv'.
Parámetros de la simulación solución 1 guardados en 'runs\08-05-2025 03_44_44 485d37\solution1\simulation_params.json'.
